In [2]:
import time
import numpy as np
import torch
import gym
from DQN import DQN_Agent,ReplayBuffer,device
from torch.utils.tensorboard import SummaryWriter
import os, shutil
from datetime import datetime
import argparse
from utils import evaluate_policy,str2bool
import metaworld

def action2box(action):
    box = np.zeros(4)
    # print("action2:",action)
    for i in range(4):
        box[i] = (action % 5)/2 - 1
        action = action // 5
    # print("2box:",box)
    return box


def box2action(box):
    # print("box2:",box)
    a = 0
    for i in range(4):
        a += (round(box[i])+1)*2 * (5**i)
    # print("2action:",a)
    # print("origbox:",box,"a2b:",action2box(a))
    return a


def load_ddqn():
    state_dim = 6
    action_dim = 625
    max_e_steps = 150
    
    algo_name = 'DDQN'

    seed = 532
    torch.manual_seed(seed)
    np.random.seed(seed)

    print('Algorithm:',algo_name,'  Env:','metaworld','  state_dim:',state_dim,'  action_dim:',action_dim,'  Random Seed:',seed, '  max_e_steps:',max_e_steps)
    print('\n')

    kwargs = {
        "state_dim": state_dim,
        "action_dim": action_dim,
        "gamma": 0.99,
        "hid_shape": (200, 200),
        "double_dqn": True,
        "env_with_dw": False,
    }
    if not os.path.exists('model'): os.mkdir('model')
    DDQN_goal1 = DQN_Agent(**kwargs)
    DDQN_goal2 = DQN_Agent(**kwargs)
    DDQN_goal3 = DQN_Agent(**kwargs)
    DDQN_goal1.load('GOAL0_DDQN_metaworld_bs512_600000.pth')
    DDQN_goal2.load('goal1/DDQN_metaworld_bs512gamma0.99nDec0.99995_450000.pth')
    DDQN_goal3.load('goal2/DDQN_metaworld_bs512gamma0.99nDec0.99995_400000.pth')
    return DDQN_goal1, DDQN_goal2, DDQN_goal3


DDQN_goal1, DDQN_goal2, DDQN_goal3 = load_ddqn()



/home/ico/Desktop/RL/RLenv/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Algorithm: DDQN   Env: metaworld   state_dim: 6   action_dim: 625   Random Seed: 532   max_e_steps: 150


loaded ./model/GOAL0_DDQN_metaworld_bs512_600000.pth
loaded ./model/goal1/DDQN_metaworld_bs512gamma0.99nDec0.99995_450000.pth
loaded ./model/goal2/DDQN_metaworld_bs512gamma0.99nDec0.99995_400000.pth


In [5]:
import numpy as np

idx = 0
idx = 1000000

def idx2state(idx, minS=np.array([-1,-1,-1,-1,-1,-1]), maxS=np.array([1,1,1,1,1,1]), steps=10):
  state = np.zeros(6)
  step_sizes = (maxS - minS)/(steps-1)
  #print(step_sizes)

  for i in range(6):
    state[i] = (idx % steps)*step_sizes[i] + minS[i]
    idx = idx // steps

  return state

In [6]:
from tqdm import tqdm

Q = np.ndarray((1000000*625,4))

DDQN = [DDQN_goal1, DDQN_goal2, DDQN_goal3]

for goal in range(3):
  for i in tqdm(range(1000000)):
#     print(idx2state(i))
    state_vec = idx2state(i)
    Q_s = DDQN[goal].select_action(state_vec, deterministic=True)
    Q[i:i+625, goal] = Q_s



 35%|██████████████████████████████████████████████████████████████                                                                                                                 | 354929/1000000 [01:42<03:07, 3447.42it/s]


KeyboardInterrupt: 

In [23]:

Q32 = Q.astype(np.float32)


In [27]:
with open('Q32_metaworld.npy', 'wb') as f:
    np.save(f, Q32)

In [10]:
def goal2box(idx):
    x = idx//2
    y = idx%2
    return np.asarray([x,y])

In [8]:
with open('Q32_metaworld.npy', 'rb') as f:
    Q32 = np.load(f)

In [11]:
action2boxarr = np.zeros((625,4))
for a in range(625):
    action2boxarr[a] = action2box(a)
    
goal2boxarr = np.zeros((4,2))
for g in range(4):
    goal2boxarr[g] = goal2box(g)

In [12]:
Q32[2*625:2*625+624,1]

array([124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 124., 124., 124., 124.,
       124., 124., 124., 124., 124., 124., 124., 12

In [13]:
# import myfm

bestactionset = set()

X = np.zeros((1000000*2*3, 6+4+2)) # 6: state dim, 4: action dim, 2: goal dim
y = np.zeros(1000000*2*3)

for stateidx in tqdm(range(1000000)):
    state = idx2state(stateidx)
    bestQ = np.asarray([Q32[stateidx*625:stateidx*625+624,i].max() for i in range(3)])
    worstQ = np.asarray([Q32[stateidx*625:stateidx*625+624,i].min() for i in range(3)])
#     print(bestQ, worstQ)
    bestOk = [False, False, False]
    worstOk = [False, False, False]
    for actionidx in range(625):
        if (False in bestOk) or (False in worstOk):
            action = action2boxarr[actionidx]
            stateactionidx = stateidx*625 + actionidx
            for goalidx in range(3):
#                 if goalidx==1:
#                     print(stateactionidx, goalidx)
#                     print(Q32[stateactionidx,goalidx],bestQ[goalidx])
#                     print(bestOk[goalidx])
                if bestOk[goalidx]==False and Q32[stateactionidx,goalidx] >= bestQ[goalidx]:
#                     print(Q32[stateactionidx,goalidx], bestQ[goalidx])
#                     print(bestOk)
                    goal = goal2boxarr[goalidx]
                    X[stateidx*2*3 + 0*3 + goalidx] = np.concatenate((state, action, goal))
#                     print(X[stateidx*2*3 + 0*3 + goalidx])
                    y[stateidx*2*3 + 0*3 + goalidx] = Q32[stateactionidx,goalidx]
                    bestOk[goalidx] = True
#                     bestactionset.add(actionidx)
                    
#                     print(bestOk, goalidx)
                if worstOk[goalidx]==False and Q32[stateactionidx,goalidx] <= worstQ[goalidx]:
                    goal = goal2boxarr[goalidx]
                    X[stateidx*2*3 + 1*3 + goalidx] = np.concatenate((state, action, goal))
                    y[stateidx*2*3 + 1*3 + goalidx] = Q32[stateactionidx,goalidx]
                    worstOk[goalidx] = True
            
            

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [02:46<00:00, 5988.84it/s]


In [140]:
len(bestactionset)

532

In [82]:
# %pip install myfm
import myfm

fm = myfm.MyFMRegressor(rank=10)
fm.fit(X,y,group_shapes=[6,4,2])

alpha = 0.01 w0 = 22.44 : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:23<00:00,  6.23s/it]


NameError: name 'fm' is not defined

In [16]:
X_goal3 = np.zeros((100*625,12))
goal = goal2boxarr[3]
for stateidx in tqdm(range(100)):
    state = idx2state(stateidx)
    for actionidx in range(625):
        action = action2boxarr[actionidx]
        X_goal3[stateidx*625 + actionidx] = np.concatenate((state, action, goal))
        


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 731.22it/s]


In [48]:
def state2idx(state):
    s = 0
    for i in range(6):
        s += (state[i]+1)*9/2*(10**i)
    return int(np.round(s))

In [56]:
state2idx([0.7777,
 -1,-1,-1,-1,-1])

8

In [103]:
step = 2/9
intervals = [-1 + step*i for i in range(10)]
goal = goal2boxarr[3]
def bestaction(state):
    X_goal3333 = np.zeros((625,12))
    for i in range(6):
        index = int(np.round((state[i] +1) / step))
        state[i] = np.clip(-1 + step*index, -1,1)
    stateidx = state2idx(state)
    for actionidx in range(625):
        action = action2boxarr[actionidx]
        X_goal3333[actionidx] = np.concatenate((state,action,goal))
    Qs = fm.predict(X_goal3333)
    bestactionnn = np.argmax(Qs)
    return bestactionnn

In [92]:
import random
for s in range(100):
    i = random.randint(0,1000000)
    print(bestaction(idx2state(i)))

600
620
4
4
4
4
620
620
620
620
600
4
4
4
620
24
600
24
4
24
4
600
24
4
620
620
4
4
620
4
620
620
4
4
4
24
620
620
600
620
4
24
620
4
24
24
4
620
620
4
4
620
4
620
620
4
4
24
4
4
620
24
620
4
4
4
4
600
600
620
600
24
24
4
620
24
4
24
620
24
24
4
24
620
620
4
4
4
600
4
4
620
620
600
4
4
620
4
4
4


In [17]:
y_goal3 = fm.predict(X_goal3)

In [19]:
len(y_goal3)

62500

In [120]:
def evaluate_policy(env, turns = 3,state_dim=None):
    scores = 0
    positive_eps = 0
    for j in range(turns):
        env.set_task(mt1.train_tasks[j+1])
        s, done, ep_r, steps = env.reset(), False, 0, 0
        s = s[:state_dim]
        for i in range(150):
            # Take deterministic actions at test time
            a = action2box(bestaction(s))
#             a = env.action_space.sample()
            s_prime, r, done, info = env.step(a)
            s_prime = s_prime[:state_dim]
            ep_r += r
            steps += 1
            s = s_prime
#             print(r)
        scores += ep_r
        print(ep_r)
        if ep_r > 0:
            positive_eps += 1
    return scores/turns, positive_eps

In [121]:
mt1 = metaworld.MT1('push-v1')  # Construct the benchmark, sampling tasks
env = mt1.train_classes['push-v1']()  # Create an environment with task `pick_place`
task = mt1.train_tasks[1]
env.set_task(task)  # Set task
env._last_rand_vec[3] = 0.09   #-0.09 or 0.09
env._last_rand_vec[4] = 0.89    #0.81 or 0.89
# eval_env = gym.make(EnvName[EnvIdex])
eval_env = env
state_dim = 6
action_dim = 625
max_e_steps = 150
print(evaluate_policy(eval_env, turns=49, state_dim=6))

-86.57635420552101
-85.78947059502578
-87.7948504854141
-87.07344669973742
-87.228162268395
-87.01291098158117
-85.6010619429071
-87.0487496126518
-84.16555952039907
-89.3089934147323
-89.1736273250817
-83.63304845357278


KeyboardInterrupt: 